# 4.1 The Metacircular Evaluator

In [4]:
;; allow access to Scheme version of apply under a different name
;; ONLY execute once per jupyter session to avoid using custom apply instead of Scheme
(define apply-in-underlying-scheme apply)

In [ ]:
(define (eval exp env)
 (newline)(display "Evalutating: ")(display exp)(newline)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((quoted? exp)
         (text-of-quotation exp))
        ;; assignment/definition recursively call eval to comput
        ;; new value associated with a variable
        ((assignment? exp)
         (eval-assignment exp env))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))
        ;; transform into applicable procedure within environment
        ((lambda? exp)
         (eval-lambda exp env))
        ;; evaluate sequence of expressions in order
        ((begin? exp)
         (eval-begin exp env))
        ;; case analysis -> nest of if expressions which are then evaluated
        ((cond? exp)
         (eval-cond exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        ((let? exp)
         (eval-let exp env))
        ((for? exp) ;; ex 4.9
         (eval-for exp env))
        ((while? exp) ;; ex 4.9
         (eval-while exp env))
        (else
         (error "Unkown expression
                type: EVAL" exp))))



(define (apply procedure arguments)
  (cond ((primitive-procedure? procedure)
         (apply-primitive-procedure
          procedure
          arguments))
        ;; sequentially eval each expression in procedure body
        ;; extend base environment to include new frame binding
        ;; params of procedure to the arguments it is applied to
        ((compound-procedure? procedure)
         (eval-sequence
          (procedure-body procedure)
          (extend-environment
           (procedure-parameters
            procedure)
           arguments
           (procedure-environment
            procedure))))
        (else
         (error "Unkown procedure
                type: APPLY"
                procedure))))

;; produce list of arguments to which a procedure is to be applied
;; evaluate each operand -> return list of results
(define (list-of-values exps env)
  (if (no-operands? exps)
      '()
      (cons (eval (first-operand exps) env)
            (list-of-values
             (rest-operands exps)
             env))))

(define (eval-if exp env)
  (if (true? (eval (if-predicate exp) env))
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

;; eval a sequence of expressions in order
;; return value of final expression
(define (eval-sequence exps env)
  (cond ((last-exp? exps)
         (eval (first-exp exps) env))
        (else
         (eval (first-exp exps) env)
         (eval-sequence (rest-exps exps)
                        env))))

;; assign value to variable
(define (eval-assignment exp env)
  (set-variable-value!
   (assignment-variable exp)
   (eval (assignment-value exp) env)
   env)
  'ok)

;; define a variable
(define (eval-definition exp env)
  (define-variable!
   (definition-variable exp)
   (eval (definition-value exp) env)
   env)
  'ok)

(define (eval-begin exp env)
  (eval-sequence
   (begin-actions exp)
   env))

(define (eval-cond exp env)
  (eval (cond->if exp) env))

(define (eval-lambda exp env)
  (make-procedure
   (lambda-parameters exp)
   (lambda-body exp)
   env))


;; expressions

(define (self-evaluating? exp)
  (cond ((number? exp) #t)
        ((string? exp) #t)
        (else #f)))

(define (variable? exp) (symbol? exp))

;; quotations: (quote <text-of-quotation>)
(define (quoted? exp)
  (tagged-list? exp 'quote))
(define (text-of-quotation exp)
  (cadr exp))

(define (tagged-list? exp tag)
  (if (pair? exp)
      (eq? (car exp) tag)
      #f))

;; assignments: (set! <var> <value>)
(define (assignment? exp)
  (tagged-list? exp 'set!))
(define (assignment-variable exp)
  (cadr exp))
(define (assignment-value exp) (caddr exp))

;; definitions: (define <var> <value>)
;; OR (define (<var> <param-1> ... <param-n>) <body>)
(define (definition? exp)
  (tagged-list? exp 'define))
(define (definition-variable exp)
  (if (symbol? (cadr exp))
      (cadr exp)
      (caadr exp)))
(define (definition-value exp)
  (if (symbol? (cadr exp))
      (caddr exp)
      (make-lambda
       (cdadr exp) ; formal parameters
       (cddr exp)))) ; body

;; (make-definition 'my-val '() 1)
;; -> (define myval 1)
;; (make-definition 'my-func` '(x y) (display (* x y))
;; -> (define (my-func x y) (display (*x y)))
(define (make-definition identifier params body)
  (if (null? params)
      (list 'define identifier body)
      (list 'define (cons identifier params) body)))

;; lambda expressions: (lambda (<param-1> ... <param-n>) <body>)
(define (lambda? exp)
  (tagged-list? exp 'lambda))
(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))
(define (make-lambda parameters body)
  (cons 'lambda (cons parameters body)))

;; conditionals: (if <predicate> <consequent> <alternative>?)
;; alternative is optional -> default to false
(define (if? exp) (tagged-list? exp 'if))
(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp)
  (if (not (null? (cdddr exp)))
      (cadddr exp)
      'false)) ;; default to false if no alternative
(define (make-if predicate
                 consequent
                 alternative)
  (list 'if
        predicate
        consequent
        alternative))

;; begin: (begin <exp-1> ... <exp-n>)
;; package a sequence of expressions into a single expression
(define (begin? exp)
  (tagged-list? exp 'begin))
(define (begin-actions exp) (cdr exp))
(define (last-exp? seq) (null? (cdr seq)))
(define (first-exp seq) (car seq))
(define (rest-exps seq) (cdr seq))
(define (sequence->exp seq)
  (cond ((null? seq) seq)
        ((last-exp? seq) (first-exp seq))
        (else (make-begin seq))))
(define (make-begin seq) (cons 'begin seq))

;; procedure application: any compound expression not defined as an expression type above
;; (<operator> <operand-1> ... <operand-n>)
(define (application? exp) (pair? exp))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
(define (no-operands? ops) (null? ops))
(define (first-operand ops) (car ops))
(define (rest-operands ops) (cdr ops))

;; cond: (cond ((<predicate-1> <result-1> ... <predicate-n> <result-1n) (else <result>))
;; implemented as nested if expressions
(define (cond? exp)
  (tagged-list? exp 'cond))
(define (cond-clauses exp) (cdr exp))
(define (cond-else-clause? clause)
  (eq? (cond-predicate clause) 'else))
(define (cond-predicate clause)
  (car clause))
(define (cond-actions clause)
  (cdr clause))
(define (cond->if exp)
  (expand-clauses (cond-clauses exp)))
(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (make-if (cond-predicate first)
                     (sequence->exp
                      (cond-actions first))
                     (expand-clauses
                      rest))))))

## 4.1
Nested `let`s will force the operands to be evaluated in the desired order. The `let` must be nested as multiple assignments in a single let will use the underlying Scheme order of evaluation:
```scheme
(let ((foo exp1) ((bar exp2)))
  ...)
```
Is equivalent to:
```scheme
((lambda (foo bar)...) exp1 exp2)
```

In [6]:
(define (list-of-values-lr exps env)
  (if (no-operands? exps)
      '()
      (let ((left (eval (first-operand exps) env)))
        (let ((right (list-of-values-lr (rest-operands exps) env)))
          (cons left right)))))

(define (list-of-values-rl exps env)
  (if (no-operands? exps)
      '()
      (let ((right (list-of-values-lr (rest-operands exps) env)))
        (let ((left (eval (first-operand exps) env)))
          (cons right left)))))

# 4.1.2 Representing Expressions

## 4.2
1. Moving the clause for procedure applications to before the clause for assignments produces the following for `(define x 3)`:
```scheme
(eval '(define x 3) env)
(application? '(define x 3)) => true
(apply (eval 'define env) (list-of-values ('x 3) env))
(eval 'define env)
(variable? 'define) => true
(lookup-variable-value 'define env) ERROR
```
2. Changing procedure applcations to start with `call` requires changing the `application?` predicate and `operator`/`operand` selectors:
```scheme
(define (application? exp)
  (tagged-list? exp 'call))
(define (operator exp) (cadr exp))
(define (operand exp) (cddr exp))
```

## 4.3

In [7]:
(define (eval-data-directed exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp env))
        ((get (car exp))
         ((get (car exp)) exp env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

(define (install-eval-data-directed)
  (put 'quote text-of-quotation)
  (put 'set! eval-assignment) 
  (put 'define eval-definition)
  (put 'if eval-if)
  (put 'lambda eval-lambda)
  (put 'begin eval-begin)
  (put 'cond eval-cond)
  'done)

## 4.4
#### `and`
- Expressions evaluated left->right
- Any expression evaluates to false -> return false
    - Remaining expressions **not** evaluated
- All expressions evaluate to true -> return value of final expression
- No expressions -> return true

#### `or`
- Expressions evaluated left->right
- Any expression evalues to true -> return that value
    - Remaining expressions **not** evaluated
- All expressions evaluate to false -> return false
- No expressions -> return false

In [8]:
;; extend to include boolean values
(define (self-evaluating? exp)
  (or (number? exp)
      (string? exp)
      (boolean? exp)))

(define (and? exp)
  (tagged-list? exp 'and))
(define (or? exp)
  (tagged-list? exp 'or))
(define (boolean-exps exp)
  (cdr exp))

(define (eval-and exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env)
           (eval-exps (rest-exps exp)))
          (else #f)))
  (eval-exps (boolean-exps exp)))

(define (eval-or exp env)
  (define (eval-exps exp)
    (cond ((last-exp? exp)
           (eval (first-exp exps) env))
          ((eval (first-exp exps) env) #t)
          (else (eval-exps (rest-exps exp)))))
  (eval-exps (boolean-exps exp)))


;; OR with derived expressions using nested if
(define (eval-and exp env)
  (eval (and->if (boolean-exps exp) env) env))
(define (eval-or exp env)
  (eval (or->if (boolean-exps exp) env) env))

(define (and->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               (and->if (rest-exps exp) env)
               #f))))

(define (or->if exp env)
  (cond ((last-exp? exp)
         (eval (first-exp exp) env))
        (else (make-if
               (first-exp exp)
               #t
               (or->if (rest-exps exp) env)))))


## 4.5
Extend `cond` to support `(<test> => <recipient>)` syntax.

In [9]:
(define (cond-arrow-form? clause)
  (eq? (cadr clause) '=>))
(define (cond-recipient clause)
  (caddr clause))

;; ((<predicate-clauses>) (<result-clauses>)) -> (<result-clauses>)
;; ((<test>) => <recipient>) -> (<recipient> (<test>))
(define (cond-consequent clause predicate)
  (if (cond-arrow-form? clause)
      (list (cond-recipient clause) predicate)
      (sequence->exp (cond-actions clause))))

(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (let ((predicate (cond-predicate first)))
              (make-if predicate
                       (cond-consequent first predicate)
                       (expand-clauses rest)))))))

## 4.6

In [10]:
(define (let? exp)
  (tagged-list? exp 'let))
(define (let-definitions exp)
  (cadr exp))
(define (let-parameters exp)
  (map car (let-definitions exp)))
(define (let-exps exp)
  (map cadr (let-definitions exp)))
(define (let-body exp)
  (cddr exp))

(define (let->combination exp env)
  (cons (make-lambda (let-parameters exp)
                     (let-body exp))
        (let-exps exp)))

(define (eval-let exp env)
  (eval (let->combination exp) env))

## 4.7
Nested `let` expressions have 2 properties which facilitate rewriting `let*`:
- Variables in each `let` are evaluated *before* the nested `let` that follows it
- Each `let` will be evaluated within the environment of the preceding `let`
    - Variables will therefore be accessible in the nested `let`

These are equivalent:
```scheme
(let* ((x 3) 
       (y (+ x 2))
       (z (+ x y 5)))
  (* x z))

(let ((x 3))
  (let ((y (+ x 2)))
    (let ((z (+ x y 5)))
      (* x z))))
```
1. `(x 3)` is evaluated and the variable `x` is assigned the value of `3` within the current environment.
2. `(y (+ x 2))` is evaluated within the same environment as step 1. As such, the value of `x` is already bound to `3` and is used to assign `y` to `5` in the current environment.
3. `(z (+ x y 5))` is evalued, since `x` and `y` are bound to values within the environment, `z` is assigned to `13` in the current environment.
4. `(* x z)` is evaluated within the same environment, returning `(* 3 13) => 39`

Having previously implemented `let`, using `(eval (let*->nested-lets exp) env)` within `eval` is sufficient to handle `let*` as the transformation in `let*->nested-lets` produces `let` expressions which `eval` is able to handle.

In [11]:
(define (let*? exp)
  (tagged-list? exp 'let*))
(define (let*-definitions exp)
  (cadr exp))
(define (let*-body exp)
  (caddr exp))

(define (let*->nested-lets exp)
  (define (make-let definitions)
    (cond ((last-exp? definitions)
           (append (list 'let (list (first-exp definitions)))
                   (let-body-exp))
           (else (list
                  'let
                  (list (first-exp definitions))
                  (make-let (rest-exps definitions)))))))
  (make-let (let*-let*-definitions)))

(define (eval-let* exp env)
  (eval (let*->nested-lets exp) env))

## 4.8
Named `let` -> procedure definition followed by 1 application of the procedure with initial values given by the `let`. Implement by creating a `begin` which contains the procedure definition, followed by it's application.

In [12]:
(define (named-let? exp)
  (and (let? exp)
       (variable? (cadr exp))))

(define (let-parameter-values exp) 
  (map cadr (let-definitions exp)))

;; exp is the named-let stripped of the let symbol
;; to allow original let selectors to be used
(define (named-let->combination exp)
  (let ((identifier (car exp)))
    (display identifier)
    (make-begin
     (list
      (make-definition identifier '()
            (make-lambda (let-parameters exp)
                         (let-body exp)))
      (cons identifier (let-parameter-values exp))))))

(define (let->combination exp env)
  (if (named-let? exp)
      (named-let->combination (cdr exp))
      (cons (make-lambda (let-parameters exp)
                         (let-body exp))
            (let-exps exp))))

## 4.9

### `for`
```scheme
(for <identifier> in <sequence>) <body>)
```
For every element in `<sequence>` execute `<body>` with the symbol `<identifier>` bound to the current element in `<sequence>`. For example:
```scheme
(for x in (list 1 2 3)
     (display x)
     (newline))
; 1
; 2
; 3
```

Can be represented as sequential applications of `<body>`, each passing the next value of `<identifier>`from `<sequence>`:
```scheme
(begin
 (define (body x) (display x) (newline))
 (body 1)
 (body 2)
 (body 3))
```

1. Create selectors for `<identifier>`, `<sequence>` and `<body>`
2. Build a `lambda` statement for `<body>` which takes `<identifier>` as a parameter
3. Build a list of applications of procedure from 2, each using a value from `<sequence>` - first need to evaluate `<sequence>` to get values
4. Build a `begin` statement containing list of applications from 3.



In [13]:
(define (for? exp)
  (tagged-list? exp 'for))
(define (for-identifier exp)
  (cadr exp))
(define (for-sequence exp)
  (cadddr exp))
(define (for-body exp)
  (cddddr exp))

(define (for->combination exp env)
  (define proc (make-lambda (for-identifier exp) (for-body exp)))
  (let ((sequence-values (eval (for-sequence exp) env)))
    (make-begin
    (map (lambda (x) (cons proc x))
         sequence-values))))

(define (eval-for exp env)
  (eval (for->combination exp env) env))

In [14]:
(for->combination '(for x in '(1 2 3) (display x)(newline)) '()) ;; dummy env

Evalutating: (quote (1 2 3))


(begin ((lambda x (display x) (newline)) . 1) ((lambda x (display x) (newline)) . 2) ((lambda x (display x) (newline)) . 3))

### `while`
```scheme
(while <predicate> <body>)
```

Execute `<body>` repeatedly whilst `<predicate>` evaluates to `true`. If `<predicate>` evaluates to `false` stop iteration. For example (state mutation used for ease of example):
```scheme
(define x 1)
(while (< x 4) ((display x)(newline) (set! x (+ x 1))))
; 1
; 2
; 3
```

1. Define selectors for `<predicate>` and `<body>`.
2. Transform into a `begin` sequence of `if` statements:
    - if `<predicate>` -> execute `<body>`, evaluate `while` statement again
    - Else return `null` / empty list 

In [15]:
(define (while? exp)
  (tagged-list? exp 'while))
(define (while-predicate exp)
  (cadr exp))
(define (while-body exp)
  (caddr exp))

(define (while->combination exp)
  (make-if
   (while-predicate exp)
   (make-begin (list (while-body exp) exp))
   '()))

(define (eval-while exp env)
  (eval (while->combination exp) env))

In [16]:
(while->combination '(while (< x 4) ((display x)(newline) (set! x (+ x 1)))))

(if (< x 4) (begin ((display x) (newline) (set! x (+ x 1))) (while (< x 4) ((display x) (newline) (set! x (+ x 1))))) ())

## 4.10
Alter the Scheme syntax to use `def` instead of `define`
```scheme
(define (definition? exp)
  (tagged-list? exp 'def))
(define (make-definition))

(define (make-definition identifier params body)
  (if (null? params)
      (list 'def identifier body)
      (list 'def (cons identifier params) body)))
```
No changes to `eval` or `apply` required.

# 4.1.3 Evaluator Data Structures

In [17]:
;; anything that is not explicit false is true
(define (true? x)
  (not (eq? x #f)))
(define (false? x)
  (eq? x #f))

;; compound procedures: (procedure (<param-1>...<param-n>) <body> <env>)
(define (make-procedure parameters body env)
  (list 'procedure parameters body env))
(define (compound-procedure? p)
  (tagged-list? p 'procedure))
(define (procedure-parameters p)
  (cadr p))
(define (procedure-body p)
  (caddr p))
(define (procedure-environment p)
  (cadddr p))

;; operations on environments

;; environment: list of frames
(define (enclosing-environment env) (cdr env))
(define (first-frame env) (car env))
(define the-empty-environment '())

;; frame: pair of lists (<variables> <values>)
(define (make-frame variables values)
  (cons variables values))
(define (frame-variables frame)
  (car frame))
(define (frame-values frame)
  (cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var (car frame)))
  (set-cdr! frame (cons val (cdr frame))))

;; extend environment: add a new frame of variables + values to an environment
(define (extend-environment vars vals base-env)
  (if (= (length vars) (length vals))
      (cons (make-frame vars vals) base-env)
      (if (< (length vars) (length vals))
          (error "Too many arguments supplied"
                 vars
                 vals)
          (error "Too few arguments supplied"
                 vars
                 vals))))

;; lookup variable in environment: start at first frame, moving up enclosing
;; environments until variable found else error
(define (lookup-variable-value var env)
  (define (env-loop env)
    (define (scan vars vals)
      (cond ((null? vars)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (car vars))
             (car vals))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (if (eq? env the-empty-environment)
         (error "LOOKUP-VARIABLE-VALUE"
                "Unbound variable ~a" var)
        (let ((frame (first-frame env)))
          (scan (frame-variables frame)
                (frame-values frame)))))
  (env-loop env))

;; set variable to value in an environment: scan for variable -> set value
;; error if variable not found
(define (set-variable-value var val env)
  (define (env-loop env)
    (define (scan vars vals)
      (cond ((null? vars)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (car vars))
             (set-car! vals val))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable: SET!" var)
        (let ((frame (first-frame env)))
          (scan (frame-variables frame)
                (frame-values frame)))))
  (env-loop env))


;; define variable: change binding if variable exists in *first* frame
;; else add new binding to first frame
(define (define-variable! var val env)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (add-binding-to-frame!
              var val frame))
            ((eq? var (car vars))
             (set-car! vals val))
            (else (scan (cdr vars)
                        (cdr vals)))))
    (scan (frame-variables frame)
          (frame-values frame))))

## 4.11

In [18]:
;; frame: list of (<variable> <value>) pairs
(define (make-frame variables values)
  (map cons variables values))
(define (frame-variables frame)
  (map car frame))
(define (frame-values frame)
  (map cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var val)))

;; lookup variable in environment: start at first frame, moving up enclosing
;; environments until variable found else error
(define (lookup-variable-value var env)
  (define (env-loop env)
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (caar rest-of-frame))
             (cdar rest-of-frame))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable" var)
        (scan (first-frame env))))
  (env-loop env))

;; set variable to value in an environment: scan for variable -> set value
;; error if variable not found
(define (set-variable-value var val env)
  (define (env-loop env)
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (env-loop
              (enclosing-environment env)))
            ((eq? var (caar rest-of-frame))
             (set-cdr! (car rest-of-frame) val))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable: SET!" var)
        (scan (first-frame env))))
  (env-loop env))


;; define variable: change binding if variable exists in *first* frame
;; else add new binding to first frame
(define (define-variable! var val env)
  (let ((frame (first-frame env)))
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (add-binding-to-frame!
              var val frame))
            ((eq? var (caar rest-of-frame))
             (set-cdr! (car frame) val))
            (else (scan (cdr rest-of-frame)))))
    (scan frame)))

## 4.12

Searching for a variable in an environment is common amongst `lookup-variable-value`, `set-variable-value!` and `define-variable`. The differences between each are how to handle when a variable *is* found within a frame and when a variable is *not* found in a frame:
- `lookup-variable-value`
    - Found -> return value of variable
    - Not found -> search enclosing environment for variable
- `set-variable-value!`
    - Found -> set the value of the variable
    - Not found -> search enclosing environment for variable
- `define-variable`
    - Found -> change binding of variable
    - Not found -> add binding in first frame

Abstract the environment searching behaviour into a procedure which includes parameters which are procedures for handling the variable found/not found cases.

I have defined these abstractions for both the original representation of frames (pair of lists) and the representation from 4.11 (list of pairs).

### Original Frame Representation: pair of lists `(<variables> <values>)`

In [19]:
(define (scan-env var env found-in-frame not-found-in-frame)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (not-found-in-frame frame))
            ((eq? var (car vars))
             (found-in-frame vars vals frame))
            (else (scan (cdr vars)
                        (cdr vals)))))
  (if (eq? env the-empty-environment)
      (error "Unbound variable" var)
      (scan (frame-variables frame)
            (frame-values frame))))) 

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals frame) (car vals))
            (lambda (frame) (lookup-variable-value var (enclosing-environment env)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (vars vals frame) (set-car! vals val))
            (lambda (frame) (set-variable-value! var val (enclosing-environment env)))))

(define (define-variable! var val env)
  (scan-env var env
            (lambda (vars vals frame) (set-car! vals val))
            (lambda (frame) (add-binding-to-frame! var val frame))))

### Frame Representation from 4.11: list of `(<variable> <value>)` pairs

In [39]:
(define (scan-env var env found-in-frame not-found-in-frame)
  (let ((frame (first-frame env)))
    (define (scan rest-of-frame)
      (cond ((null? rest-of-frame)
             (not-found-in-frame frame))
            ((eq? var (caar rest-of-frame))
             (found-in-frame (car rest-of-frame)))
            (else (scan (cdr rest-of-frame)))))
    (if (eq? env the-empty-environment)
        (error "Unbound variable" var)
        (scan frame))))

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (binding) (car binding))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (binding) (set-cdr! binding val))
            (lambda (frame) (scan-env (enclosing-environment env)))))

(define (define-variable! var val env)
  (scan-env var env
            (lambda (binding) (set-car! binding val))
            (lambda (frame) (add-binding-to-frame! var val frame))))


## 4.13

`make-unbound!` should only remove the binding in the current lexical scope (first frame). Removing bindings in global scope from within a nested scope could cause errors when other scopes attempt to access that binding (defeating the purpose of scoped environments).

I have only implemented for the original representation of frames and will continue to do so from now on.

In [20]:
(define (make-unbound! var env)
  (scan-env var env
            (lambda (vars vals)
              (set-car! vars '())
              (set-car! vals '())
              #t)
            #f))

# 4.1.4 Running the Evaluator as a Program

In [51]:
(define primitive-procedures
  (list (list 'car car)
        (list 'cdr cdr)
        (list 'cons cons)
        (list 'null? null?)
        (list '+ +)
        (list '- -)
        (list '* *)
        (list '/ /)
        (list '= =))) ;; other primitives can be added here

(define (primitive-procedure-names)
  (map car primitive-procedures))

(define (primitive-procedure-objects)
  (map (lambda (proc)
         (list 'primitive (cadr proc)))
       primitive-procedures))

(define (primitive-procedure? proc)
  (tagged-list? proc 'primitive))

(define (primitive-implementation proc)
  (cadr proc))

(define (apply-primitive-procedure proc args)
  (apply-in-underlying-scheme
   (primitive-implementation proc) args))

(define (setup-environment)
  (let ((initial-env
         (extend-environment
          (primitive-procedure-names)
          (primitive-procedure-objects)
          the-empty-environment)))
    ;; bind true/false for use in evaluating expressions
    (define-variable! 'true #t initial-env)
    (define-variable! 'false #f initial-env)
    initial-env))

(define the-global-environment
  (setup-environment))



(define input-prompt ";;; M-Eval input:")
(define output-prompt ";;; M-Eval value:")

(define (driver-loop)
  (prompt-for-input input-prompt)
  (let ((input (read)))
    (let ((output
           (eval input
                 the-global-environment)))
      (announce-output output-prompt)
      (user-print output)))
  (driver-loop))

(define (prompt-for-input string)
  (newline) (newline)
  (display string) (newline))

(define (announce-output string)
  (newline) (display string) (newline))

;; avoid printing environment part of a compound procedure
;; environment may be very large or contain cycles
(define (user-print object)
  (if (compound-procedure? object)
      (display
       (list 'compound-procedure
             (procedure-parameters object)
             (procedure-body object)
             '<procedure-env>))
      (display object)))

## 4.14
The system version of `map` will use the system version of `apply`, not the version defined in our evaluator. It will therefore be unable to interpret the arguments passed to it as they will not be in the underlying Scheme representation. 

For example, evaluating:
```scheme
(map + '(1 2) '(3 4))
```

`apply` will call `list-of-values` to obtain the values required for a procedure application:
```scheme
(apply (eval (operator exp) env)
       (list-of-values (operands exp) env))
```

`list-of-values` returns a list made by calling `eval` with each operand in the expression. `+` is defined as a primitive procedure in the evaluator, therefore it is returned as:
```scheme
'(primitive +)
```

Scheme's built in `map` will be unable to use this as it's evaluator will treat it as a `quote` literal and won't execute the intended primitive procedure `+`.

Eva's `map` works because it uses the `apply` in our evaluator, which will 'understand' the values passed to it.

# 4.1.5 Data as Programs

## 4.15

Given a procedure `p`, `try` will run forever if `(halts? p p)` evaluates to true.

For `(try try)` to run forever, `(halts? try try)` must evaluate to true, which means that `(try try)` cannot run forever.

Similarly, for `(try try)` to halt, `(halts? try try)` must evaluate to false, which means that `(try try)` cannot halt.

Both scenarios violate the intended behaviour of `halts?`.

# 4.1.6 Internal Definitions

## 4.16

In [34]:
;; calysto-scheme doesn't have filter built in
;; name filter-seq to prevent collision in other scheme implementations
(define (filter-seq pred? seq)
  (if (null? seq)
      '()
      (let ((first (car seq))
            (rest (cdr seq)))
        (if (pred? first)
            (cons first (filter-seq pred? rest))
            (filter-seq pred? rest)))))

;; 1
;; using abstractions from ex 4.12hh
(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals frame)
              (let ((val (car vals)))
                (if (eq? val '*unassigned*)
                    (error "unassigned variable" (car vars))
                    val)))
            (lambda (frame) (lookup-variable-value var (enclosing-environment env)))))

;; 2
;; find all defines in proc-body
;; create let statement assigning all define vars to '*unassigned*
;; create set! statements assgning define expressions to vars
(define (scan-out-defines proc-body)
  (let ((defs (body-definitions proc-body)))
    (if (null? defs)
        proc-body
        (append (list 'let (unassigned-defines defs))
                      (unassigned-defines-assignments defs)
                      (scanned-out-body proc-body)))))

(define (body-definitions body)
  (filter-seq definition? body))

(define (scanned-out-body body)
  (filter-seq (lambda (exp) (not (definition? exp))) body))

(define (unassigned-defines defines)
  (map (lambda (var) (list var '*unassigned*))
       (map definition-variable defines))) 

(define (unassigned-defines-assignments defines)
  (map (lambda (def)
         (list 'set!
               (definition-variable def)
               (definition-value def)))
       defines))

;; 3
;; installed into make-procedure
;; only called once when the lambda is first evaluated
;; installing in procedure-body would call scan-out-defines
;; each time the procedure is called
(define (make-procedure parameters body env)
  (list 'procedure parameters (scan-out-defines body) env))

## 4.17
(excluding environment diagrams)

Untransformed procedure:
```scheme
(lambda ⟨vars⟩
  (define u ⟨e1⟩)
  (define v ⟨e2⟩)
  ⟨e3⟩)
```

Transformed procedure:
```scheme
(lambda ⟨vars⟩
  (let ((u '*unassigned*)
        (v '*unassigned*))
    (set! u ⟨e1⟩)
    (set! v ⟨e2⟩)
    ⟨e3⟩))
```

When the untransformed procedure is evaluated, the internal definitions (`v` and `u`) are added to the current frame - `<e3>` is evaluated within this frame.

When the transformed procedure is evaluated, the `let` expression is transformed into a `lambda` expression. This `lambda` creates a *new* frame within the environment of outer `lambda` definition.

This change in environment structure can never make a differnce in the behaviour of a correct program because the additional frame produced by the `let` expression during evaluation *isoloates* it's internal definitions from the outer environment.

To implement the 'simultaneous' scope rule for internal definitions without constructing an extra frame, the interpreter would need to *enforce* that all internal definitions must come *before* the body of the procedure. For example:

```scheme
(define (f x)
  ;; defines before procedure body
  (define foo (lambda (y) (bar x y)))
  (define bar (lambda (a b) (+ (* a b) a)))
  (foo (+ x 3))) 
(f 5)
;; => 45
```

```scheme
(define (f x)
  (foo (+ x 3))) 
  ;; defines after procedure body
  (define foo (lambda (y) (bar x y)))
  (define bar (lambda (a b) (+ (* a b) a)))
(f 5)
;; => Error internal defines cannot appear after procedure body
```

## 4.18

### Original procedure
```scheme
(define (solve f y0 dt)
  (define y (integral (delay dy) y0 dt))
  (define dy (stream-map f y))
  y)
```

### Transformed as shown in exercise using multiple `let` expressions
```scheme
(define (solve f y0 dt)
  (let ((y '*unassigned*)
        (dy '*unassigned*))
    (let ((a (integral (delay dy) y0 dt))
          (b (stream-map f y)))
      (set! y a)
      (set! dy b))
    y))
```

When evaluate, both `let` expressions are transformed in `lambda`s:
```scheme
(define (solve f y0 dt)
  ((lambda (y dy)
     ((lambda (a b)
        (set! y a)
        (set! dy b))
      (integral (delay dy) y0 dt)
      (stream-map f y))
     y)
   '*unassigned* '*unassigned*))
```

The arguments `y` and `dy` for the outer lambda are both `'*unassigned*` and will be used in the arguments passed to the inner lambda:
```scheme
;; inner lambda arguments
(integral (delay '*unassigned*) y0 dt)
(stream-map f '*unassigned*))
```

The evaluation of these will result in an error and so this transformation **will not** work.

### Transformed as shown in the text using a single `let` expression
```scheme
(define (solve f y0 dt)
  (let ((y '*unassigned*)
        (dy '*unassigned*))
    (set! y (integral (delay dy) y0 dt))
    (set! dy (stream-map f y))
    y))
```

When evaluated, the `let` expression is transformed into a `lambda`:
```scheme
(define (solve f y0 dt)
  ((lambda (y dy)
     (set! y (integral (delay) y0 dt))
     (set! dy (stream-map f y))
     y)
   '*unassigned* '*unassigned*))
```

This transformation will work as expected, producing 'simultaneous' evaluation of the original internal procedures.

## 4.19

```scheme
(let ((a 1))
  (define (f x)
    (define b (+ a x))
    (define a 5)
    (+ a b))
  (f 10))
```

Eva's view of `a` and `b` being *truly* simultaneous is the ideal behaviour. However, implementing such behaviour is non-trivial as it requires finding the correct order to evaluate mutually recursive and dependent procedure definitions to produce the effect of simultaneous evaluation.

In the simple case given in the example this may not be a problem, however with more complex relationships between definitions it could become practically intractable.

Potential Implementation:
1. Scan-out all definitions as in 4.16 and set all variables to `*unassigned*`
2. Search through definitions to find those which **do not** depend on any unassigned variables
3. Evaluate these procedures, updating any variables in the current environment
4. Repeat steps 2 and 3 untill all definitions have been evaluated. Raise an error if at any point all definitions are dependent on an unassigned variable.



## 4.20

### 1

In [23]:
;; transform letrec into a combination in the same manner as scan-out-defines
(define (letrec? exp)
  (tagged-list? exp 'letrec))
(define (letrec-declarations exp)
  (cadr exp))
(define (letrec-variable declaration)
  (car declaration))
(define (letrec-value declaration)
  (cadr declaration))
(define (letrec-body exp)
  (caddr exp))

(define (letrec->combination exp)
  (let ((decs (letrec-declarations exp)))
    (if (null? decs)
      exp
      (append (list 'let (unassigned-declarations decs))
                    (unassigned-declaration-assignments decs)
                    (letrec-body exp)))))

(define (unassigned-declarations declarations)
  (map (lambda (var) (list var '*unassigned*))
       (map letrec-variable declarations)))

(define (unassigned-declaration-assignments declarations)
  (map (lambda (dec)
         (list 'set!
               (letrec-variable dec)
               (letrec-value dec)))
       declarations))

In [24]:
;; example
(define f
  '(letrec
      ((even?
        (lambda (n)
          (if (= n 0)
              true
              (odd? (- n 1)))))
       (odd?
        (lambda (n)
          (if (= n 0)
              false
              (even? (- n 1))))))
    (even? 2)))

(letrec->combination f)

(let ((even? *unassigned*) (odd? *unassigned*)) (set! even? (lambda (n) (if (= n 0) true (odd? (- n 1))))) (set! odd? (lambda (n) (if (= n 0) false (even? (- n 1))))) even? 2)

### 2

Louis' idea won't work for the same reasons explained in exercise 4.17.

```scheme
(define (f x)
  (let ((even?
            (lambda (n)
              (if (= n 0)
                  true
                  (new-odd? (- n 1)))))
           (odd?
            (lambda (n)
              (if (= n 0)
                  false
                  (new-even? (- n 1))))))
    ((even? x)))) ;; replaced <rest of body> placeholder
```

This is then transformed into a `lambda` with the values from the declarations provided as arguments:
```scheme
(define (f x)
  ((lambda (even? odd?) 
     (even? x)))    
   (lambda (n)
     (if (= n 0)
         true
         (odd? (- n 1))))
   (lambda (n)
     (if (= n 0)
         false
         (even? (- n 1))))))
```

This produces an error due to the arguments `even?` and `odd?` to the `lambda` being unassigned.

## 4.21

### 1

In [25]:
(define (factorial-lambdas n)
 ((lambda (fact) (fact fact n))
  (lambda (ft k)
    (if (= k 1)
        1
        (* k (ft ft (- k 1)))))))

(map factorial-lambdas '(1 2 3 4 5 6 7 8 9 10))

(1 2 6 24 120 720 5040 40320 362880 3628800)

In [26]:
(define (fib-lambdas n)
  ((lambda (fact) (fact fact n))
     (lambda (ft k)
       (cond ((= k 0) 0)
             ((= k 1) 1)
             (else
              (+ (ft ft (- k 1))
                 (ft ft (- k 2))))))))

(map fib-lambdas '(1 2 3 4 5 6 7 8 9 10))

(1 1 2 3 5 8 13 21 34 55)

### 2

In [27]:
(define (f x)
  ((lambda (even? odd?)
     (even? even? odd? x))
   (lambda (ev? od? n)
     (if (= n 0)
         #t
         (od? ev? od? (- n 1))))
   (lambda (ev? od? n)
     (if (= n 0)
         #f
         (ev? ev? od? (- n 1))))))

(map f '(1 2 3 4 5 6 7 8 9 10))

(#f #t #f #t #f #t #f #t #f #t)

# 4.1.7 Separating Syntactic Analysis from Execution

In [50]:
;; renamed to make comparison in ex 4.24 simpler
(define (eval-pre-analysis exp env) ((analyze exp) env))

;; return a procedure to be applied in an environment
;; to evaluate an expression without repeating syntactic analysis 
(define (analyze exp)
  (cond ((self-evaluating? exp)
         (analyze-self-evaluating exp))
        ((quoted? exp) 
         (analyze-quoted exp))
        ((variable? exp) 
         (analyze-variable exp))
        ((assignment? exp) 
         (analyze-assignment exp))
        ((definition? exp) 
         (analyze-definition exp))
        ((if? exp) 
         (analyze-if exp))
        ((lambda? exp) 
         (analyze-lambda exp))
        ((begin? exp) 
         (analyze-sequence 
          (begin-actions exp)))
        ((cond? exp) 
         (analyze (cond->if exp)))
        ((application? exp) 
         (analyze-application exp))
        ((let? exp) ;; ex 4.22
         (analyze-let exp))
        (else
         (error "Unknown expression 
                 type: ANALYZE" 
                exp))))

;; return procedure ignoring environment that returns the expression
(define (analyze-self-evaluating exp)
  (lambda (env) exp))

;; extract text of quotation only once
(define (analyze-quoted exp)
  (let ((qval (text-of-quotation exp)))
    (lambda (env) qval)))

;; looking up variables must still be done in execution phase
;; as it is environment dependant
(define (analyze-variable exp)
  (lambda (env)
    (lookup-variable-value exp env)))

;; analyze assignment-value only once
;; setting the variable is still deferred until execution
(define (analyze-assignment exp)
  (let ((var (assignment-variable exp))
        (vproc (analyze
                (assignment-value exp))))
    (lambda (env)
      (set-variable-value!
       var (vproc env) env)
      'ok)))

;; analyze definition-value only once
;; defining the variable is still deferred until execution
(define (analyze-definition exp)
  (let ((var (definition-variable exp))
        (vproc (analyze
                (definition-value exp))))
    (lambda (env)
      (define-variable! var (vproc env) env)
      'ok)))

;; analyze predicate, consequent and alternative at analysis time
(define (analyze-if exp)
  (let ((pproc (analyze (if-predicate exp)))
        (cproc (analyze (if-consequent exp)))
        (aproc (analyze (if-alternative exp))))
    (lambda (env)
      (if (true? (pproc env))
          (cproc env)
          (aproc env)))))

;; analyze lambda body only once at analysis time
(define (analyze-lambda exp)
  (let ((vars (lambda-parameters exp))
        (bproc (analyze-sequence
                (lambda-body exp))))
    (lambda (env)
      (make-procedure vars bproc env))))

;; analyze each expression in the sequence into execution procedures
;; combine execution procedures to procue an execution procedure 
;; that sequentially calls each individual exection procedure in the environment
(define (analyze-sequence exps)
  ;; execute proc1 *then* proc2
  (define (sequentially proc1 proc2)
    (lambda (env) (proc1 env) (proc2 env)))
  ;; sequentially call each procedure in the sequence
  (define (loop first-proc rest-procs)
    (if (null? rest-procs)
        first-proc
        (loop (sequentially first-proc
                            (car rest-procs))
              (cdr rest-procs))))
  (let ((procs (map analyze exps)))
    (if (null? procs)
        (error "ANALYZE" "Empty sequence"))
    (loop (car procs) (cdr procs))))

;; analyze operator and operands
;; return execution procedure that:
;; - calls operator execution procedure -> obtain procedure to be applied
;; - calls operand execution procedufes -> obtain arguments
;; pass these to execute-application
(define (analyze-application exp)
  (let ((fproc (analyze (operator exp)))
        (aprocs (map analyze (operands exp))))
    (lambda (env)
      (execute-application
       (fproc env)
       (map (lambda (aproc) (aproc env))
       aprocs)))))

(define (execute-application proc args)
  (cond ((primitive-procedure? proc)
         (apply-primitive-procedure proc args))
        ((compound-procedure? proc)
         ;; body already analyzed -> execute in extended environment
         ;; containing params + values for procedure
         ((procedure-body proc)
          (extend-environment
           (procedure-parameters proc)
           args
          (procedure-environment proc))))
        (else (error "EXECUTE-APPLICATION" "Unkown procedure type ~a" proc))))

## 4.22

`let` is evaluated via a syntactic transformation to a `lambda` expression using `let->combination`. Since `lambda` expressions are already supported in the new evaluator (`analyze-lambda`), `let` expression  supported is achieved by calling `analyze` on the result of `let->combination`.

In [29]:
;; installed in analyze above
(define (analyze-let exp)
  (analyze (let->combination exp)))

## 4.23

Alyssa P.Hacker's version of `analyze-sequence`:
```scheme
(define (analyze-sequence exps)
  (define (execute-sequence procs env)
    (cond ((null? (cdr procs))
           ((car procs) env))
          (else ((car procs) env)
                (execute-sequence
                 (cdr procs) env))))
  (let ((procs (map analyze exps)))
    (if (null? procs)
        (error "ANALYZE" "Empty sequence"))
    (lambda (env)
      (execute-sequence procs env))))
```

The version of `analyze-sequence` in the text produces a `lambda` at *analysis* time which executes the analyzed procedures in the given sequence.

Alyssa's version of `analyze-sequence` produces a `lambda` at *analysis* time that when called at *runtime* uses `execute-sequence` to execute the analyzed procedures in the given sequence.

Both versions analyze each procedure in the sequence at *analysis* time, however Alyssa's version produces the procedure to actually execute the analyzed procedures at *runtime* - thus on every execution of the sequence.

For a single procedure, Alyssa's version produces an execution procedure that loops over a single element list of analyzed procedures and executes it in the current environment:
```scheme
(lambda (env)
  (execute-sequence (list analyzed-proc) env))

;; leads to 
((analyzed-proc) env))
```

For a single procedure, the version in the text simply returns the execution procedure of the analyzed procedure:
```scheme
(analyze-sequence (list proc))

(lambda (env) (<analyzed-proc>))
```

For two procedures, Alyssa's version produces an exection procedure that loops over a two element list of analyzed procedures and executes each one in the current environment:
```scheme
(analyze-sequence (list proc1 proc2))
(lambda (env)
  (execute-sequence (list analyzed-proc1 analyzed-proc2) env))
;; leads to
((analyzed-proc1) env)
(execute-sequence list(analyzed-proc2) env)
leads to
((analyzed-proc2) env)
```

For two procedures, the version in the text produces a lambda that contains sequential exeuctions of each analyzed procedure:
```scheme
(analyze-sequence (list proc1 proc2))
(lambda (env)
  (analyzed-proc1 env)
  (analyzed-proc2 env))
```

## 4.24

In [30]:
;; returns the time taken to execute an expression
(define-syntax time 
  [(time ?exp) (let ((start (current-time)))
                 ?exp
                 (- (current-time) start))])

(time (fib-lambdas 15))

0.23861312866210938

In [31]:
;; times execution of a sequence of expressions
;; for both versions of eval
(define (benchmark exps)
  (define (run eval-proc)
    (let ((env (setup-environment)))
      (define (run-interpreter)
        (map (lambda (exp)
               (eval-proc exp env))
             exps))
      (time (run-interpreter))))
  (display "No pre-analysis: ") (display (run eval)) (newline)
  (display "Pre-analysis: ") (display (run eval-pre-analysis)) (newline))

### Highly Iterative Procedure

In [37]:
(benchmark
 (list
  '(define (factorial n)
     (if (= n 0) 1
         (* n (factorial (- n 1)))))
  '(factorial 20))) 

No pre-analysis: 0.11526131629943848
Pre-analysis: 0.048169851303100586


### No Iteration

In [38]:
(benchmark
 (list
  '(define (add x y)
     (+ x y))
  '(add 1 2)))

No pre-analysis: 0.005812644958496094
Pre-analysis: 0.005399227142333984


For highly iterative procedures, pre-analysis provides a significant speed increase. This is due to the procedure being analysed only once, prior to evalutaion and as such it is not analysed on every single iteration.

For non-iterative procedure, pre-analysis is actually *slightly* slower because it has to analyse the procedure and create an execution procedure before it can be exeucted. This however is a trivial example as most procedures are executed more more than *once* throughout the execution of a program.